In [26]:
# 設定影片資料夾
import os
input_folder = "video"
output_folder = "output"

In [37]:
print("目前工作目錄:", os.getcwd())

目前工作目錄: /Users/linlab/Documents


In [29]:
####影片轉檔
import os


#from moviepy.editor import VideoFileClip 爛東西，搞死我，放著紀念

format_list = ['avi','mov','wmv','flv','asf', 'mkv']

file_name=os.listdir(input_folder)
video_name=[]
for k in file_name:
    dd=k.split(".")
    if dd[-1]=="avi":
        video_name.append(k)
os.chdir(input_folder)  # 使用 Colab 要換路徑使用
#print(video_name)

out=[]
for k in video_name:
    k=k.split(".")
    out.append(k[0]+"_new")
print(out,video_name)
i=0
for k in video_name:
    os.popen("ffmpeg -i {input}  {output}.mp4".format(input = k, output = out[i]))
    i+=1
print('慘')

"""
ffmpeg -i {input} -strict -2 {output}.mp4 
如果要直接在terminal上跑，使用上面這行
"""

[] []
慘


'\nffmpeg -i {input} -strict -2 {output}.mp4 \n如果要直接在terminal上跑，使用上面這行\n'

In [31]:
####返回原本的位置
os.chdir("..")

In [32]:
####影像修正
####如不需進行鏡頭參數校正 可直截跳到下面

In [33]:
# 檔案路徑與輸出資料夾
import os
video_path = '20250213_174956_601_20250213174955.avi'  # 替換為你的 .avi 檔案路徑
output_folder = 'frames'       # 儲存幀的資料夾

In [34]:
#隨機選取照片
import cv2
import random
# 打開影片檔案
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("無法打開影片檔案")
    exit()

# 取得影片資訊
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 隨機選擇 20 個幀號
random_frames = random.sample(range(total_frames), 20)
random_frames.sort()

# 擷取並保存幀
for i, frame_no in enumerate(random_frames):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)  # 移動到指定幀
    ret, frame = cap.read()                    # 讀取幀
    if ret:
        output_path = os.path.join(output_folder, f'frame_{i + 1}.jpg')
        cv2.imwrite(output_path, frame)        # 保存幀為圖片
        print(f"保存幀 {frame_no} 到 {output_path}")
    else:
        print(f"無法讀取幀 {frame_no}")

# 釋放資源
cap.release()
print("完成！")

保存幀 60 到 frames/frame_1.jpg
保存幀 135 到 frames/frame_2.jpg
保存幀 160 到 frames/frame_3.jpg
保存幀 295 到 frames/frame_4.jpg
保存幀 303 到 frames/frame_5.jpg
保存幀 326 到 frames/frame_6.jpg
保存幀 333 到 frames/frame_7.jpg
保存幀 334 到 frames/frame_8.jpg
保存幀 338 到 frames/frame_9.jpg
保存幀 408 到 frames/frame_10.jpg
保存幀 414 到 frames/frame_11.jpg
保存幀 485 到 frames/frame_12.jpg
保存幀 584 到 frames/frame_13.jpg
保存幀 601 到 frames/frame_14.jpg
保存幀 872 到 frames/frame_15.jpg
保存幀 875 到 frames/frame_16.jpg
保存幀 917 到 frames/frame_17.jpg
保存幀 976 到 frames/frame_18.jpg
保存幀 1029 到 frames/frame_19.jpg
保存幀 1130 到 frames/frame_20.jpg
完成！


In [39]:
#利用照片進行校正


# 設定棋盤格的內部角點數量 (cols, rows)
chessboard_size = (6, 7)

#進入資料夾
os.chdir(output_folder)

import numpy as np
import glob
import cv2 as cv

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboard_size[0]*chessboard_size[1],3), np.float32)
objp[:,:2] = np.mgrid[0:chessboard_size[0],0:chessboard_size[1]].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('*.jpg')
 
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (chessboard_size[0],chessboard_size[1]), None)
 
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
 
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [40]:
####返回原本的位置
os.chdir("..")

In [41]:
#儲存參數
np.savez('data.npz', mtx=mtx, dist=dist)

In [42]:

print("相機矩陣 (Camera Matrix):\n", mtx)
print("畸變係數 (Distortion Coefficients):\n", dist)

相機矩陣 (Camera Matrix):
 [[955.91523669   0.         415.40649808]
 [  0.         627.69757974 258.04847792]
 [  0.           0.           1.        ]]
畸變係數 (Distortion Coefficients):
 [[-0.62469404  0.79207032 -0.00383659  0.01341145 -0.84251035]]


In [ ]:
#校正影片
import numpy as np

#讀取之前的參數
#with open('data.csv', mode='r', newline='') as file:
    #reader = csv.reader(file)
    #data = list(reader)
    #mtx=data[0]
    #dist=data[1]
data = np.load('data.npz')
mtx=data['mtx']
dist=data['dist']

# 確保輸出資料夾存在
os.makedirs(output_folder, exist_ok=True)

mtx=np.array(mtx, dtype=np.float32)
dist=np.array(dist)
# 取得所有影片檔案
video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4'))]

# 遍歷所有影片
for video_file in video_files:
    input_path = os.path.join(input_folder, video_file)
    output_path = os.path.join(output_folder, f"undistorted_{video_file}")

    print(f"處理影片: {video_file} -> {output_path}")

    # 讀取影片
    cap = cv2.VideoCapture(input_path)

    # 取得影片的基本資訊
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # 定義影片輸出編碼格式
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # MP4 格式
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # 計算新的相機矩陣
    new_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, 
                                                           (frame_width, frame_height), 1, 
                                                           (frame_width, frame_height))

    # 預先計算映射表
    mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, 
                                              new_mtx, (frame_width, frame_height), cv2.CV_32FC1)

    # 讀取並處理影片
    while cap.isOpened():#確定打開影片
        ret, frame = cap.read()
        if not ret:
            break  # 影片播放完畢

        # 校正畫面
        undistorted_frame = cv2.remap(frame, mapx, mapy, cv2.INTER_LINEAR)

        # 寫入校正後的影片
        out.write(undistorted_frame)

    # 釋放資源
    cap.release()
    out.release()
    cv.destroyAllWindows()
    
    print(f"✅ 已完成: {output_path}")

print(" 所有影片處理完成！")

處理影片: 20250213_174956_601_20250213174955_new.mp4 -> frames/undistorted_20250213_174956_601_20250213174955_new.mp4
✅ 已完成: frames/undistorted_20250213_174956_601_20250213174955_new.mp4
處理影片: 20250103_155020_601_20250103155020_new.mp4 -> frames/undistorted_20250103_155020_601_20250103155020_new.mp4
✅ 已完成: frames/undistorted_20250103_155020_601_20250103155020_new.mp4
處理影片: 20231122_153144_603_20231122153143_new.mp4 -> frames/undistorted_20231122_153144_603_20231122153143_new.mp4
